In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
                    .master("local")\
                    .appName("spark sql")\
                    .getOrCreate()


In [6]:
import pyspark.sql.functions as f

In [4]:
csv_df = spark.read.csv("file:///home/shital/Desktop/pyspark_program/input/StateCensusData.csv",header=True, inferSchema=True)

In [7]:
csv_df.show()

+--------------+----------+----------+--------------+
|         State|Population|AreaInSqKm|DensityPerSqKm|
+--------------+----------+----------+--------------+
| Uttar Pradesh| 199812341|    240928|           828|
|   Maharashtra| 112372972|    307713|           365|
|         Bihar| 103804637|     94163|          1102|
|Madhya Pradesh|  91347736|     88752|          1029|
|Madhya Pradesh|  72597565|    308245|           236|
|    Tamil Nadu|  72138958|    130058|           555|
|     Rajasthan|  68621012|    342239|           201|
|     Karnataka|  61130704|    191791|           319|
|       Gujarat|  60383628|    196024|           308|
|         Bihar|  49386799|    162968|           303|
|        Odisha|  41947358|    155707|           269|
|     Telangana|  35286757|    114840|           307|
|        Kerala|  33387677|     38863|           859|
|   Maharashtra|  32966238|     79714|           414|
|Madhya Pradesh|  31169272|     78438|           397|
|        Punjab|  27704236| 

In [8]:
csv_df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- AreaInSqKm: integer (nullable = true)
 |-- DensityPerSqKm: integer (nullable = true)



In [9]:
csv_df.limit(5).show()

+--------------+----------+----------+--------------+
|         State|Population|AreaInSqKm|DensityPerSqKm|
+--------------+----------+----------+--------------+
| Uttar Pradesh| 199812341|    240928|           828|
|   Maharashtra| 112372972|    307713|           365|
|         Bihar| 103804637|     94163|          1102|
|Madhya Pradesh|  91347736|     88752|          1029|
|Madhya Pradesh|  72597565|    308245|           236|
+--------------+----------+----------+--------------+



In [10]:
csv_df.select('*').show()

+--------------+----------+----------+--------------+
|         State|Population|AreaInSqKm|DensityPerSqKm|
+--------------+----------+----------+--------------+
| Uttar Pradesh| 199812341|    240928|           828|
|   Maharashtra| 112372972|    307713|           365|
|         Bihar| 103804637|     94163|          1102|
|Madhya Pradesh|  91347736|     88752|          1029|
|Madhya Pradesh|  72597565|    308245|           236|
|    Tamil Nadu|  72138958|    130058|           555|
|     Rajasthan|  68621012|    342239|           201|
|     Karnataka|  61130704|    191791|           319|
|       Gujarat|  60383628|    196024|           308|
|         Bihar|  49386799|    162968|           303|
|        Odisha|  41947358|    155707|           269|
|     Telangana|  35286757|    114840|           307|
|        Kerala|  33387677|     38863|           859|
|   Maharashtra|  32966238|     79714|           414|
|Madhya Pradesh|  31169272|     78438|           397|
|        Punjab|  27704236| 

In [11]:
csv_df.select('State','Population').limit(5).show()

+--------------+----------+
|         State|Population|
+--------------+----------+
| Uttar Pradesh| 199812341|
|   Maharashtra| 112372972|
|         Bihar| 103804637|
|Madhya Pradesh|  91347736|
|Madhya Pradesh|  72597565|
+--------------+----------+



In [12]:
csv_df.where((csv_df.DensityPerSqKm > 400) & (csv_df.State == 'Madhya Pradesh')).show()

+--------------+----------+----------+--------------+
|         State|Population|AreaInSqKm|DensityPerSqKm|
+--------------+----------+----------+--------------+
|Madhya Pradesh|  91347736|     88752|          1029|
|Madhya Pradesh|  25353081|     44212|           573|
+--------------+----------+----------+--------------+



In [13]:
csv_df.agg({'Population':'avg'}).show()

+--------------------+
|     avg(Population)|
+--------------------+
|6.3608398166666664E7|
+--------------------+



In [14]:
csv_df.groupBy('State').agg({'Population':'avg'}).orderBy('State', ascending = False).show()

+--------------+---------------+
|         State|avg(Population)|
+--------------+---------------+
| Uttar Pradesh|   1.99812341E8|
|     Telangana|    3.5286757E7|
|    Tamil Nadu|    7.2138958E7|
|     Rajasthan|    6.8621012E7|
|        Punjab|    2.7704236E7|
|        Odisha|    4.1947358E7|
|   Maharashtra|    5.6959802E7|
|Madhya Pradesh|   5.51169135E7|
|        Kerala|    3.3387677E7|
|     Karnataka|    6.1130704E7|
|       Gujarat|    6.0383628E7|
|         Bihar|    7.6595718E7|
+--------------+---------------+



In [15]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

def int_float(x):
    return float(x)

int_float_udf = udf(int_float, FloatType())
csv_df.select('State',int_float_udf('Population')).alias('Population in float').show()

+--------------+---------------------+
|         State|int_float(Population)|
+--------------+---------------------+
| Uttar Pradesh|         1.99812336E8|
|   Maharashtra|         1.12372976E8|
|         Bihar|          1.0380464E8|
|Madhya Pradesh|          9.1347736E7|
|Madhya Pradesh|          7.2597568E7|
|    Tamil Nadu|           7.213896E7|
|     Rajasthan|          6.8621008E7|
|     Karnataka|          6.1130704E7|
|       Gujarat|          6.0383628E7|
|         Bihar|            4.93868E7|
|        Odisha|           4.194736E7|
|     Telangana|          3.5286756E7|
|        Kerala|          3.3387676E7|
|   Maharashtra|          3.2966238E7|
|Madhya Pradesh|          3.1169272E7|
|        Punjab|          2.7704236E7|
|   Maharashtra|          2.5540196E7|
|Madhya Pradesh|           2.535308E7|
+--------------+---------------------+



In [16]:
csv_df.createOrReplaceTempView("data")

In [17]:
spark.sql("describe data").show()

+--------------+---------+-------+
|      col_name|data_type|comment|
+--------------+---------+-------+
|         State|   string|   null|
|    Population|      int|   null|
|    AreaInSqKm|      int|   null|
|DensityPerSqKm|      int|   null|
+--------------+---------+-------+



In [18]:
spark.sql('select * from data').show()

+--------------+----------+----------+--------------+
|         State|Population|AreaInSqKm|DensityPerSqKm|
+--------------+----------+----------+--------------+
| Uttar Pradesh| 199812341|    240928|           828|
|   Maharashtra| 112372972|    307713|           365|
|         Bihar| 103804637|     94163|          1102|
|Madhya Pradesh|  91347736|     88752|          1029|
|Madhya Pradesh|  72597565|    308245|           236|
|    Tamil Nadu|  72138958|    130058|           555|
|     Rajasthan|  68621012|    342239|           201|
|     Karnataka|  61130704|    191791|           319|
|       Gujarat|  60383628|    196024|           308|
|         Bihar|  49386799|    162968|           303|
|        Odisha|  41947358|    155707|           269|
|     Telangana|  35286757|    114840|           307|
|        Kerala|  33387677|     38863|           859|
|   Maharashtra|  32966238|     79714|           414|
|Madhya Pradesh|  31169272|     78438|           397|
|        Punjab|  27704236| 

22/03/22 12:04:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 128198 ms exceeds timeout 120000 ms
22/03/22 12:04:39 WARN SparkContext: Killing executors is not supported by current scheduler.
